# 索引类型

https://github.com/facebookresearch/faiss/wiki/Faiss-indexes

## 数据准备

In [1]:
import numpy as np

d = 64                           # dimension
nb = 10000                       # database size
np.random.seed(1234)             # make reproducible
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.

# Summary of methods

Method|Class name|index_factory|Main parameters|Bytes/vector|Exhaustive|Comments
---|---|---|---|---|---|---
L2精确查询|IndexFlatL2|"Flat"|d|4*d|yes|brute-force
Inner Product精确查询|IndexFlatIP|"Flat"|d|4*d|yes|also for cosine (normalize vectors beforehand)
Hierarchical Navigable Small World graph exploration|IndexHNSWFlat|"HNSWx,Flat"|d, M|4*d + 8 * M|no	
Inverted file with exact post-verification|IndexIVFFlat|"IVFx,Flat"|quantizer, d, nlists, metric|4*d|no|Take another index to assign vectors to inverted lists
Locality-Sensitive Hashing (binary flat index)|IndexLSH|-|d, nbits|nbits/8|yes|optimized by using random rotation instead of random projections
Scalar quantizer (SQ) in flat mode|IndexScalarQuantizer|"SQ8"|d|d|yes|4 bit per component is also implemented, but the impact on accuracy may be inacceptable
Product quantizer (PQ) in flat mode|IndexPQ|"PQx"|d, M, nbits|M (if nbits=8)|yes	
IVF and scalar quantizer|IndexIVFScalarQuantizer|"IVFx,SQ4" "IVFx,SQ8"|quantizer, d, nlists, qtype|SQfp16: 2 * d, SQ8: d or SQ4: d/2|no|there are 2 encodings: 4 bit per dimension and 8 bit per dimension
IVFADC (coarse quantizer+PQ on residuals)|IndexIVFPQ|"IVFx,PQy"|quantizer, d, nlists, M, nbits|M+4 or M+8|no|the memory cost depends on the data type used to represent ids (int or long), currently supports only nbits <= 8
IVFADC+R (same as IVFADC with re-ranking based on codes)|IndexIVFPQR|"IVFx,PQy+z"|quantizer, d, nlists, M, nbits, M_refine, nbits_refine|M+M_refine+4 or M+M_refine+8|no	

The index can be constructed explicitly with the class constructor, or by using index_factory.

# Flat indexes

`Flat` indexes 把向量集合编码成固定大小的code，保存在`ntotal * code_size`字节的数组中.

检索时, 解码所有已索引的向量，与查询向量进行比较. For the IndexPQ the comparison is done in the compressed domain, which is faster.

> 为什么起名叫Flat，表明没有压缩，与PQ相对

### Supported operations
Flat 索引类似 C++ 向量. They do not store vector ids, since in many cases sequential numbering is enough. Therefore:
- 不支持`add_with_id` (but they can be wrapped in an IndexIDMap to add that functionality).
- 支持高效直接向量存取(with `reconstruct`和`reconstruct_n`)
- 支持向量移除（remove方法）. Note that this shrinks the index and changes the numbering.

### Vectors encodings
The available encodings are (from least to strongest compression):
- no encoding at all (IndexFlat): the vectors are stored without compression;
- 16-bit float encoding (IndexScalarQuantizer with QT_fp16): the vectors are compressed to 16-bit floats, which may cause some loss of precision;
- 8/6/4-bit integer encoding (IndexScalarQuantizer with QT_8bit/QT_6bit/QT_4bit): vectors quantized to 256/64/16 levels;
- PQ encoding (IndexPQ): vectors are split into sub-vectors that are each quantized to a few bits (usually 8). See the example below.

In [2]:
import faiss

index = faiss.IndexFlatL2(d)
index.add(xb)
D, I = index.search(xb[:5], 5)
print(I)

[[  0 393 363  78 924]
 [  1 555 277 364 617]
 [  2 304 101  13 801]
 [  3 173  18 182 484]
 [  4 288 370 531 178]]


In [3]:
index = faiss.IndexScalarQuantizer(d, 4)
index.train(xb)
index.add(xb)
D, I = index.search(xb[:5], 5)
print(I)

[[  0 393 363  78 924]
 [  1 555 277 364 617]
 [  2 304 101  13 801]
 [  3 173  18 182 484]
 [  4 288 370 531 178]]


In [4]:
# 当数据库向量是标准化的，计算返回的distance就是余弦相似度
d = 512          #维数
n_data = 2000   
np.random.seed(0) 
data = []
mu = 3
sigma = 0.1
for i in range(n_data):
    data.append(np.random.normal(mu, sigma, d))
data = np.array(data).astype('float32')

index = faiss.IndexFlatIP(d)
index.add(data)
D, I = index.search(data[:5], 5)
print(I)

[[1562   27  169  681 1262]
 [  27 1562  681  169 1262]
 [1562   27  169  681 1262]
 [1562   27  681  169 1262]
 [1562   27  681  169 1262]]


# Cell-probe methods (IndexIVF* indexes)

A typical way to speed-up the process at the cost of loosing the guarantee to find the nearest neighbor is to employ a partitioning technique such as k-means. The corresponding algorithms are sometimes referred to as cell-probe methods.

We use a partition-based method based on Multi-probing (a reminiscent variant of best-bin KD-tree).

- The feature space is partitioned into nlist cells.
- The database vectors are assigned to one of these cells thanks to a hashing function (in the case of k-means, the assignment to the centroid closest to the query), and stored in an inverted file structure formed of nlist inverted lists.
- At query time, a set of nprobe inverted lists is selected
- The query is compared to each of the database vector assigned to these lists.
Doing so, only a fraction of the database is compared to the query: as a first approximation, this fraction is nprobe/nlist, but this approximation is usually under-estimated because the inverted lists have not equal lengths. The failure case appears when the cell of the nearest neighbor of a given query is not selected.

The constructor takes an index as a parameter (the quantizer or coarse quantizer), which is used to do the assignment to the inverted lists. The query is searched in this index, and the returned vector id(s) are the inverted list(s) that should be visited.

### Cell probe method with a flat index as coarse quantizer
Typically, one would use a Flat index as coarse quantizer. The train method of the IndexIVF adds the centroids to the flat index. The nprobe is specified at query time (useful for measuring trade-offs between speed and accuracy).

NOTE: As a rule of thumb, denoting by n the number of points to be indexed, a typical way to select the number of centroids is to aim at balancing the cost of the assignment to the centroids (nlist * d for a plain k-means) with the number of distance computations performed when parsing the inverted lists (in the order of nprobe / nlist * n * C, where the constant accounts for the uneven distribution of the list and the fact that a single vector comparison is done more efficiently when done by batch with centroids, say C=10 to give an idea). This leads to a number of centroids of the form nlist = C * sqrt (n).

### Other types of coarse quantizers
In some contexts it is beneficial to use other types of quantizers, for example a GPU based quantizer, a MultiIndexQuantizer or a HNSW based quantizer.

在一些索引类型中，需要一个Flat index作为粗量化器，如IndexIVFFlat,在训练的时候会将类中心保存在Flat index中，在add和search阶段，会首先判定将其落入哪个类空间。在search阶段，nprobe参数需要调整以权衡检索精度与检索速度。  
实验表明，对高维数据，需要维持比较高的nprobe数值才能保证精度。

### Encoding of vectors in an IndexIVF
The elements of inverted lists are encoded vectors (+ the corresponding vector id). The encoding is mainly to make the vectors more compact. Those elements are just scanned sequentially, and the search function returns the top-k smallest distances seen so far.

The supported codes are the same as for the Flat index, just convert the name of the index class by inserting IVF: IndexFlat becomes IndexIVFFlat.

为了加速索引过程，经常采用划分子类空间（如k-means）的方法，虽然这样无法保证最后返回的结果是完全正确的。先划分子类空间，再在部分子空间中搜索的方法，就是cell-probe方法。  
具体流程为：  
1) 数据集空间被划分为n个部分，在k-means中，表现为n个类；  
2) 每个类中的向量保存在一个倒排表中，共有n个倒排表；  
3) 查询时，选中nprobe个倒排表；  
4) 将这几个倒排表中的向量与查询向量作对比。  
在这种方法中，只需要排查数据库中的一部分向量，大约只有nprobe/n的数据，因为每个倒排表的长度并不一致（每个类中的向量个数不一定相等）。

与LSH的优劣
LSH也是一种cell-probe方法，与其相比，LSH有一下一点不足：  
1）LSH需要大量的哈希方程，会带来额外的内存开销；  
2）哈希函数不适合输入数据。

In [6]:
nlist = 50
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist)
index.train(xb)
index.add(xb)
D, I = index.search(xb[:5], 5)
print(I)

[[   0  363  491 1124  608]
 [   1  277  617  756   88]
 [   2  101   13  134  225]
 [   3   18  182  484  527]
 [   4  370  531  175   18]]


In [ ]:
# 倒排表乘积量化, IVFADC (coarse quantizer+PQ on residuals))
M = 8
nbits = 4
nlist = 50
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFPQ(quantizer, d, nlist, M, nbits)
index.train(xb)
index.add(xb)
D, I = index.search(xb[:5], 5)
print(I)

# IndexHNSW variants
The Hierarchical Navigable Small World indexing method is based on a graph built on the indexed vectors. At search time, the graph is explored in a way that converges to the nearest neighbors as quickly as possible. The IndexHNSW uses a flat index as underlying storage to quickly access the database vectors and abstract the compression / decompression of vectors. HNSW depends on a few important parameters:

- M is the number of neighbors used in the graph. A larger M is more accurate but uses more memory
- efConstruction is the depth of exploration at add time
- efSearch is the depth of exploration of the search

### Supported encodings
IndexHNSW supports the following Flat indexes: IndexHNSWFlat (no encoding), IndexHNSWSQ (scalar quantizer), IndexHNSWPQ (product quantizer), IndexHNSW2Level (two-level encoding).

### Supported operations
In addition to the restrictions of the Flat index HNSW uses, HNSW does not support removing vectors from the index. This would destroy the graph structure.

In [ ]:
index = faiss.IndexHNSWFlat(d,16)
index.add(xb)
D, I = index.search(xb[:5], 5)
print(I)

# IndexLSH and its relationship with cell-probe methods
The most popular cell-probe method is probably the original Locality Sensitive Hashing method referred to as [E2LSH] (http://www.mit.edu/~andoni/LSH/). However this method and its derivatives suffer from two drawbacks:

- They require a lot of hash functions (=partitions) to achieve acceptable results, leading to a lot of extra memory. Memory is not cheap.
- The hash function are not adapted to the input data. This is good for proofs but leads to suboptimal choice results in practice.

### Binary LSH codes
In Faiss, the IndedLSH is just a Flat index with binary codes. The database vectors and query vectors are hashed into binary codes that are compared with Hamming distances.

In Python, the (improved) LSH index is constructed and search as follows

```python
n_bits = 2 * d
lsh = faiss.IndexLSH (d, n_bits)
lsh.train (x_train)
lsh.add (x_base)
D, I = lsh.search (x_query, k)
```

NOTE: The algorithm is not vanilla-LSH, but a better choice. Instead of set of orthogonal projectors is used if n_bits <= d, or a tight frame if n_bits > d.

In [ ]:
nbits = 2 * d
index = faiss.IndexLSH(d, nbits)
index.train(xb)
index.add(xb)
D, I = index.search(xb[:5], 5)
print(I)

# Indexes based on Product Quantization codes
In Python, a product quantizer is defined by:
```python
m = 16                                   # number of subquantizers
n_bits = 8                               # bits allocated per subquantizer
pq = faiss.IndexPQ (d, m, n_bits)        # Create the index
pq.train(x_train)                       # Training
pq.add(x_base)                          # Populate the index
D, I = pq.search(x_query, k)            # Perform a search
```
The number of bits n_bits must be equal to 8, 12 or 16. The dimension d should be a multiple of m

### Inverted file with PQ refinement
`IndexIVFPQ`是对large-scale搜索最有用的索引结构. It is used like
```python
coarse_quantizer = faiss.IndexFlatL2 (d)
index = faiss.IndexIVFPQ(coarse_quantizer, d, ncentroids, code_size, 8)
index.nprobe = 5
```
See the chapter about IndexIVFFlat for the setting of ncentroids. `code_size`通常是2的幂, 4到64之间. 像`IndexPQ`, `d`是`m`的倍数.

In [ ]:
M = 8 
nbits = 6  #只能是8， 12， 16
index = faiss.IndexPQ(d, M, nbits)
index.train(xb)
index.add(xb)
D, I = index.search(xb[:5], 5)
print(I)